In [1]:
import os
import sys
import numpy as np
import pandas as pd

src_path = os.path.abspath(os.path.join(os.path.curdir, "src"))
sys.path.append(src_path)

from src.dataset import read_dataset

In [2]:
df = read_dataset()

For collaborative filtering, we need to choose only 1 type of "item" to recommand to the user. For that, we choose to recommand artists to user. 

In [5]:
user_artists_df = df.groupby(['user_id', 'artist_id']).agg(
    timestamp = pd.NamedAgg(column='timestamp', aggfunc='max'),
).reset_index()
user_track_df = df.groupby(['user_id', 'track_id']).agg(
    timestamp = pd.NamedAgg(column='timestamp', aggfunc='max'),
).reset_index()

user_artists_df = user_artists_df.rename(columns={'artist_id': 'item_id'})
user_track_df = user_track_df.rename(columns={'track_id': 'item_id'})

user_item_df = pd.concat([user_artists_df, user_track_df])

user_item_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184954 entries, 0 to 154894
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     --------------   -----              
 0   user_id    184954 non-null  object             
 1   item_id    184954 non-null  object             
 2   timestamp  184954 non-null  datetime64[us, UTC]
dtypes: datetime64[us, UTC](1), object(2)
memory usage: 5.6+ MB


In [6]:
from src.cf_mf import MatrixFactorization

mf = MatrixFactorization(df=user_item_df,
                         batch_size=1024,
                         embedding_size=64,
                         learning_rate=0.001,
                         regularization=1e-5)
mf.fit(epochs=10, val_epoch=1)

KeyError: 'artist_id'